## Import all our libraries

In [110]:
import pandas as pd
import numpy as np
import twint
import nest_asyncio
nest_asyncio.apply()
from nltk.corpus import stopwords
from nltk import word_tokenize
from  nltk import FreqDist
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import PorterStemmer
import string

In [111]:
df_all_samples = pd.read_csv('all_samples.csv')

In [112]:
df_all_samples.columns

Index(['Unnamed: 0', 'hashtags', 'label', 'like_count', 'reply_count',
       'retweet_count', 'tweet'],
      dtype='object')

In [113]:
df_all_samples.drop(columns='Unnamed: 0', inplace=True)

In [114]:
df_all_samples.head()

,hashtags,label,like_count,reply_count,retweet_count,tweet
0,[politics],1,0.0,0.0,0.0,#politics NH gun shop owner gifts Trump semi-a...
1,"[ISIS, targets, iceisis, opiceisis]",1,0.0,0.0,0.0,RT @CtrlSec: Targeted #ISIS accounts https://t...
2,[],0,42.0,0.0,9.0,16-year-old environmental activist Greta Thunb...
3,['#moleg'],0,6.0,1.0,1.0,"Thank God for @repdottieb4mo Under her watch,..."
4,[],0,4.0,0.0,0.0,The Rise of Purpose Education: A Recipe for Fu...


## Testing some patterns for cleaning.....

In [141]:
df_all_samples['tweet'][1]

'RT @CtrlSec: Targeted #ISIS accounts https://t.co/YbhmYQgJZv https://t.co/gZIfZKUgOU https://t.co/jhasANJmDN #targets #iceisis #opiceisis'

In [153]:
df_all_samples['tweet'][2]

'16-year-old environmental activist Greta Thunberg is sick of shouldering the burden for adults: “I should be in school on the other side of the ocean... You have stolen my dreams and my childhood with your empty words, and yet I’m one of the lucky ones."  http://instyle.io/fSHwey7\xa0'

In [119]:
t

'RT @CtrlSec: Targeted #ISIS accounts https://t.co/YbhmYQgJZv https://t.co/gZIfZKUgOU https://t.co/jhasANJmDN #targets #iceisis #opiceisis'

In [154]:
df_all_samples['tweet'][3]

'Thank God for @repdottieb4mo  Under her watch, #MOLeg 110 remains safe & secure from the leftist liberal horde. 😎  https://twitter.com/repdottieb4mo/status/1176507943182843904\xa0…'

In [120]:
import re

In [144]:
pat = r'(RT)*(@\w+)*(#\w+)*(\w+)'

In [149]:
m = re.search(pat, t)

In [152]:
m.groups()

(None, None, None, 'RT')

In [142]:
df_all_samples['hashtags'][1]

'[ISIS, targets, iceisis, opiceisis]'

In [ ]:
## 
## Terst regex - keep this
##
# \b(RT)\b.*(@\w+).*(https:\/\/\w+\.?\w+\/?\w+)(\s+|\w+)*.*(#\w+)

In [157]:
t

'RT @CtrlSec: Targeted #ISIS accounts https://t.co/YbhmYQgJZv https://t.co/gZIfZKUgOU https://t.co/jhasANJmDN #targets #iceisis #opiceisis'

In [155]:
pat2 = r'@\S+|https?://\S+'

In [189]:
len(t)

137

In [176]:
pat1 = r'@\S+|https?://\S+'

In [177]:
t1 = re.sub(pat1, '', t)

In [178]:
t1

'RT  Targeted #ISIS accounts    #targets #iceisis #opiceisis'

In [179]:
pat2 = r'#\w+\b'

In [180]:
t2 = re.sub(pat2, '', t1)

In [182]:
t2.strip()

'RT  Targeted  accounts'

In [164]:
t2 = re.sub(pat2, '', t)

In [185]:
pat3 = r'\bRT\b'

In [187]:
t3 = re.sub(pat3, '', t2)

In [188]:
t3.strip()

'Targeted  accounts'

In [161]:
pat4 = r'@\S+|https?://\S+#\w+\b'

In [162]:
re.sub(pat4, '', t)

'RT  Targeted #ISIS accounts https://t.co/YbhmYQgJZv https://t.co/gZIfZKUgOU https://t.co/jhasANJmDN #targets #iceisis #opiceisis'

In [232]:
test_df = df_all_samples[0:10]

In [233]:
test_df.head()

,hashtags,label,like_count,reply_count,retweet_count,tweet
0,[politics],1,0.0,0.0,0.0,#politics NH gun shop owner gifts Trump semi-a...
1,"[ISIS, targets, iceisis, opiceisis]",1,0.0,0.0,0.0,RT @CtrlSec: Targeted #ISIS accounts https://t...
2,[],0,42.0,0.0,9.0,16-year-old environmental activist Greta Thunb...
3,['#moleg'],0,6.0,1.0,1.0,"Thank God for @repdottieb4mo Under her watch,..."
4,[],0,4.0,0.0,0.0,The Rise of Purpose Education: A Recipe for Fu...


## Important code to do basic cleaning of tweets using Regex patterns.  Cleaning is applied iteratively and sequentially, which seems to have good results.  Here we set up our core patterns and our "cleaner" lambda functions to be used with apply() below.

In [240]:
pat1 = r'@\S+|https?://\S+'
clean_1 = lambda x: re.sub(pat1, '', x)

pat2 = r'#\w+\b'
clean_2 = lambda x: re.sub(pat2, '', x)

pat3 = r'\bRT\b'
clean_3 = lambda x: re.sub(pat3, '', x)

pat4 = r'@\S+|https?://\S+#\w+\b'
clean_4 = lambda x: re.sub(pat4, '', x)

clean_5 = lambda x: x.strip()

cleaners = [clean_1, clean_2, clean_3, clean_4, clean_5]

## Here we iterate over our list of "cleaners" and apply them to the tweet column in our dataframe, saving the cleaned tweets in a separate column, so we can do later analysis.

In [242]:
# Get our first pass of cleaned tweets into a new column
# and then continue to clean data in that "cleaned_tweets"
# column
df_all_samples['cleaned_tweets'] = df_all_samples.tweet.apply(clean_1)
for cleaner in cleaners[1:]:
    df_all_samples['cleaned_tweets'] = df_all_samples.cleaned_tweets.apply(cleaner)

In [244]:
df_all_samples.head(200)

,hashtags,label,like_count,reply_count,retweet_count,tweet,cleaned_tweets
0,[politics],1,0.0,0.0,0.0,#politics NH gun shop owner gifts Trump semi-a...,NH gun shop owner gifts Trump semi-automatic r...
1,"[ISIS, targets, iceisis, opiceisis]",1,0.0,0.0,0.0,RT @CtrlSec: Targeted #ISIS accounts https://t...,Targeted accounts
2,[],0,42.0,0.0,9.0,16-year-old environmental activist Greta Thunb...,16-year-old environmental activist Greta Thunb...
3,['#moleg'],0,6.0,1.0,1.0,"Thank God for @repdottieb4mo Under her watch,...","Thank God for Under her watch, 110 remains ..."
4,[],0,4.0,0.0,0.0,The Rise of Purpose Education: A Recipe for Fu...,The Rise of Purpose Education: A Recipe for Fu...
5,NaN,1,0.0,0.0,0.0,Here is my dilemma. One half of me wants you a...,Here is my dilemma. One half of me wants you a...
6,['#drtsexbook'],0,13.0,0.0,9.0,“This book is part of my activism and advocacy...,“This book is part of my activism and advocacy...
7,[FF],1,0.0,0.0,0.0,RT @flippable_org: #FF Some great news sources...,Some great news sources and curators who educa...
8,[],0,12.0,6.0,2.0,Yes. We are the “snowflakes” here. It’s the @T...,Yes. We are the “snowflakes” here. It’s the t...
9,"[MAGA, ThankYou]",1,0.0,0.0,0.0,RT @justjo7656: @POTUS Good Luck tonight! Stan...,Good Luck tonight! Stand strong! We the people...


In [246]:
df_all_samples.cleaned_tweets[24]

'PM Morrison🐍 absurdly attacks journalists\' & Opposition questions over Liberal million-dollar fundraiser MP💣Gladys Liu\'s ties to foreign💰govt as "racist"  🚨Propaganda arm of foreign govt praises Morrison for repeating their line  Worried yet?   \xa0…'

In [248]:
et = df_all_samples.cleaned_tweets[24]

In [247]:
emotic = r'(\u00a9|\u00ae|[\u2000-\u3300]|\ud83c[\ud000-\udfff]|\ud83d[\ud000-\udfff]|\ud83e[\ud000-\udfff])'

In [250]:
re.sub(emotic, '', et)

'PM Morrison🐍 absurdly attacks journalists\' & Opposition questions over Liberal million-dollar fundraiser MP💣Gladys Liu\'s ties to foreign💰govt as "racist"  🚨Propaganda arm of foreign govt praises Morrison for repeating their line  Worried yet?   \xa0'

In [251]:
RE_EMOJI = re.compile('[\U00010000-\U0010ffff]', flags=re.UNICODE)


In [252]:
re.sub(RE_EMOJI, '', et)

'PM Morrison absurdly attacks journalists\' & Opposition questions over Liberal million-dollar fundraiser MPGladys Liu\'s ties to foreigngovt as "racist"  Propaganda arm of foreign govt praises Morrison for repeating their line  Worried yet?   \xa0…'

In [253]:
RE_EMOJI = re.compile('[\U00010000-\U0010ffff]', flags=re.UNICODE)
pat6 = RE_EMOJI
clean_6 = lambda x: re.sub(pat6, '', x)

In [254]:
df_all_samples['cleaned_tweets'] = df_all_samples.cleaned_tweets.apply(clean_6)

In [257]:
df_all_samples.head(200)

,hashtags,label,like_count,reply_count,retweet_count,tweet,cleaned_tweets
0,[politics],1,0.0,0.0,0.0,#politics NH gun shop owner gifts Trump semi-a...,NH gun shop owner gifts Trump semi-automatic r...
1,"[ISIS, targets, iceisis, opiceisis]",1,0.0,0.0,0.0,RT @CtrlSec: Targeted #ISIS accounts https://t...,Targeted accounts
2,[],0,42.0,0.0,9.0,16-year-old environmental activist Greta Thunb...,16-year-old environmental activist Greta Thunb...
3,['#moleg'],0,6.0,1.0,1.0,"Thank God for @repdottieb4mo Under her watch,...","Thank God for Under her watch, 110 remains ..."
4,[],0,4.0,0.0,0.0,The Rise of Purpose Education: A Recipe for Fu...,The Rise of Purpose Education: A Recipe for Fu...
5,NaN,1,0.0,0.0,0.0,Here is my dilemma. One half of me wants you a...,Here is my dilemma. One half of me wants you a...
6,['#drtsexbook'],0,13.0,0.0,9.0,“This book is part of my activism and advocacy...,“This book is part of my activism and advocacy...
7,[FF],1,0.0,0.0,0.0,RT @flippable_org: #FF Some great news sources...,Some great news sources and curators who educa...
8,[],0,12.0,6.0,2.0,Yes. We are the “snowflakes” here. It’s the @T...,Yes. We are the “snowflakes” here. It’s the t...
9,"[MAGA, ThankYou]",1,0.0,0.0,0.0,RT @justjo7656: @POTUS Good Luck tonight! Stan...,Good Luck tonight! Stand strong! We the people...


In [258]:
df_all_samples.to_csv('cleaned_all_samples.csv')

hashtags                       [ElectoralCollege, HillaryLostAgain]
label                                                             1
like_count                                                        0
reply_count                                                       0
retweet_count                                                     0
tweet             RT @jojoh888: Liberal tears 😭😭😭💦💦💦 they'll kee...
cleaned_tweets              Liberal tears  they'll keep flowing    
Name: 480, dtype: object